In [18]:
!nvidia-smi

Wed Sep 29 16:00:53 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 472.12       Driver Version: 472.12       CUDA Version: 11.4     |
|-------------------------------+----------------------+----------------------+
| GPU  Name            TCC/WDDM | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ... WDDM  | 00000000:01:00.0  On |                  N/A |
|  0%   47C    P8    20W / 250W |    906MiB / 11264MiB |      9%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [22]:
USE_CUDA = torch.cuda.is_available()
print(USE_CUDA)

False


In [19]:
#Set GPU
import os
import random
os.environ['CUDA_VISIBLE_DEVICES'] = '0' #Set GPU number

In [20]:
#Load Packages
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import imageio
from IPython.display import HTML

import torch
import torch.nn as nn
import torch.nn.parallel
import torch.backends.cudnn as cudnn
import torch.optim as optim
import torch.utils.data
import torchvision.datasets as dset
import torchvision.transforms as transforms
import torchvision.utils as vutils
from torchvision.utils import make_grid

In [5]:
if not os.path.exists('./checkpoint'):
  os.mkdir('./checkpoint')
if not os.path.exists('./dataset'):
  os.mkdir('./dataset')
if not os.path.exists('./img'):
  os.mkdir('./img')
if not os.path.exists('./img/real'):
  os.mkdir('./img/real')
if not os.path.exists('./img/fake'):
  os.mkdir('./img/fake')

In [6]:
# visualize the first image from the torch tensor
def vis_image(image):
    plt.imshow(image[0].detach().cpu().numpy(),cmap='gray')
    plt.show()


In [7]:
def save_gif(training_progress_images, images):
    '''
        training_progress_images: list of training images generated each iteration
        images: image that is generated in this iteration
    '''
    img_grid = make_grid(images.data)
    img_grid = np.transpose(img_grid.detach().cpu().numpy(), (1, 2, 0))
    img_grid = 255. * img_grid 
    img_grid = img_grid.astype(np.uint8)
    training_progress_images.append(img_grid)
    imageio.mimsave('./img/training_progress.gif', training_progress_images)
    return training_progress_images

In [8]:
# visualize gif file
def plot_gif(training_progress_images, plot_length=10):
    plt.close()
    fig = plt.figure()
    
    total_len = len(training_progress_images)
    for i in range(plot_length):
        im = plt.imshow(training_progress_images[int(total_len/plot_length)*i])
        plt.show()

In [9]:
def save_image_list(dataset, real):
  if real:
    base_path = './img/real'
  else:
    base_path = './img/fake'

  dataset_path = []

  for i in range(len(dataset)):
    save_path = f'{base_path}/image_{i}.png'
    dataset_path.append(save_path)
    vutils.save_image(dataset[i], save_path)

  return base_path

In [11]:
dataroot = './tmp_block/'

dataset = dset.ImageFolder(root = dataroot, transform = transforms.Compose([transforms.Resize(512), transforms.CenterCrop(512), transforms.ToTensor(), transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5,0.5))]))

dataloader = torch.utils.data.DataLoader(dataset, batch_size = 128, shuffle = True, num_workers = 2)
device = torch.device('cuda:0')

In [12]:
#Intialize convolutional, convolutional-transpose from a Normal distribution (0, 0.02) and batch normalization layers to meet this criteria
def weights_init(m):
  classname = m.__class__.__name__
  if classname.find('Conv') != -1:
    nn.init.normal_(m.weight.data, 0.0, 0.02)
  elif classname.find('BatchNorm') != -1:
    nn.init.normal_(m.weight.data, 1.0, 0.02)
    nn.init.constant_(m.bias.data, 0)

In [13]:
# Num of channels in training image (In this case, RGB-3)
nc = 3
# Size of z latent vector (Size of generator input)
nz = 100
# Size of feature maps in generator
ngf = 512
# Size of feature maps in discriminator
ndf = 512

num_epochs = 20
lr = 0.0002
# Beta1 hyperparam for Adam potimizers
beta1 = 0.5
# Number of GPUs available.
ngpu = 1

In [14]:
######################---------Genrator----------#######################
class Generator(nn.Module):
  def __init__(self, ngpu):
    super(Generator, self).__init__()
    self.ngpu = ngpu
    self.main = nn.Sequential(
        #Input latent Z
        nn.ConvTranspose2d(nz, ngf*8, 4, 1, 0, bias = False),
        nn.BatchNorm2d(ngf*8),
        nn.ReLU(True),
        #1st state (ngf*8 *4 *4)
        nn.ConvTranspose2d(ngf*8, ngf*4, 4, 2, 1, bias = False),
        nn.BatchNorm2d(ngf*4),
        nn.ReLU(True),
        # state size. (ngf*4) x 8 x 8
        nn.ConvTranspose2d( ngf * 4, ngf * 2, 4, 2, 1, bias=False),
        nn.BatchNorm2d(ngf * 2),
        nn.ReLU(True),
        # state size. (ngf*2) x 16 x 16
        nn.ConvTranspose2d( ngf * 2, ngf, 4, 2, 1, bias=False),
        nn.BatchNorm2d(ngf),
        nn.ReLU(True),
        #state size(ngf) X 32 X 32
        nn.ConvTranspose2d(ngf, nc, 4, 2, 1, bias = False),
        nn.Tanh()
    )
  def forward(self, input):
    output = self.main(input)
    return output

In [21]:
##Create Generator
netG = Generator(ngpu).cuda()

if (device.type == 'cuda') and (ngpu>1):
  netG = nn.DataParallel(netG, list(range(ngpu)))

netG.apply(weights_init)

print(netG)

AssertionError: Torch not compiled with CUDA enabled